<!-- ## Store the RPPG Result and the Graph of the Subject folders

Will store the RPPG signal (raw) and then the pre-processed one (v0.1) and the image comparasion between the GT -->

In [170]:
## Import Dependencies
import numpy as np
import mediapipe as mp
import pandas as pd
import cv2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import matplotlib.pyplot as plt
import scipy
import os
import scipy.signal

In [171]:
base_model="../Models/blaze_face_short_range.tflite"
# base_model="../Models/face_landmarker.task"

<!-- ## Define the POS method
Pos Method by Wenjin Wang -->

In [172]:
# ## Core method POS 
# def POS(signal, **kargs):
#     """
#     POS method on CPU using Numpy.

#     The dictionary parameters are: {'fps':float}.

#     Wang, W., den Brinker, A. C., Stuijk, S., & de Haan, G. (2016). Algorithmic principles of remote PPG. IEEE Transactions on Biomedical Engineering, 64(7), 1479-1491. 
#     """
#     """
#     eps: A small constant (10^-9) used to prevent division by zero in normalization steps.
#     X: The input signal, which is a 3D array where:
#     e: Number of estimators or regions in the frame (like different parts of the face).
#     c: Color channels (3 for RGB).
#     f: Number of frames.
#     w: Window length, determined by the camera's frame rate (fps). For example, at 20 fps, w would be 32 (which corresponds to about 1.6 seconds of video).
#     """
#     eps = 10**-9
#     X = signal
#     e, c, f = X.shape # Number of estimators, color channels, and frames
#     w = int(1.6 * kargs['fps']) # Window length in frames

#     """
#     P: A fixed 2x3 matrix used for the projection step. It defines how to transform the color channels (RGB) into a new space.
#     Q: This is a stack of the matrix P repeated e times, where each P corresponds to an estimator (region of interest) in the video.
#     """
#     P = np.array([[0, 1, -1], [-2, 1, 1]]) ## Pulse Rate Matricies
#     Q = np.stack([P for _ in range(e)], axis = 0)

#     """
#     H: A matrix to store the estimated heart rate signal over time for each estimator.
#     n: The current frame in the sliding window.
#     m: The start index of the sliding window (calculating which frames are part of the current window).
#     """
#     H = np.zeros((e, f))
#     for n in np.arange(w, f):
#         # Start index of sliding window 
#         m = n - w + 1

#         """
#         Temporal Normalization (Equation 5 from the paper): This step ensures that the signal is invariant to global lighting changes and other noise factors.
#         """
#         Cn = X[:, :, m:(n+1)]
#         M = 1.0 / (np.mean(Cn, axis = 2) + eps)
#         M = np.expand_dims(M, axis=2) # shape [e, c, w]
#         Cn = np.multiply(Cn, M)

#         """
#         Projection (Equation 6 from the paper): This step transforms the RGB values into a space where the signal from blood flow (heart rate) is more distinct.
#         """
#         S = np.dot(Q, Cn)
#         S = S[0, :, :, :]
#         S = np.swapaxes(S, 0, 1) 

#         """
#         Tuning (Equation 7 from the paper): This step adjusts the projected components to make the heart rate signal clearer.
#         """
#         S1 = S[:, 0, :]
#         S2 = S[:, 1, :]
#         alpha = np.std(S1, axis=1) / (eps + np.std(S2, axis=1))
#         alpha - np.expand_dims(alpha, axis=1)
#         Hn = np.add(S1, alpha * S2)
#         Hnm = Hn - np.expand_dims(np.mean(Hn, axis=1), axis=1)

#         """
#         Overlap-Adding (Equation 8 from the paper): This step combines the processed signals from each frame to form the final output heart rate signal.
#         """
#         H[:, m:(n + 1)] = np.add(H[:, m:(n + 1)], Hnm)  # Add the tuned signal to the output matrix

#     return H

<!-- ### Setup Mediapipe ROI Capture

Setup the functions and properties for doing the RPPGs -->

In [173]:
# ## Create faceDetector Object
# base_options = python.BaseOptions(model_asset_path=base_model)
# FaceDetector = mp.tasks.vision.FaceDetector
# FaceDetectorOptions = mp.tasks.vision.FaceDetectorOptions
# VisionRunningMode = mp.tasks.vision.RunningMode
# options = FaceDetectorOptions(
#     base_options=base_options,
#     running_mode = VisionRunningMode.IMAGE,
# )
# detector = vision.FaceDetector.create_from_options(options)

In [174]:
# ## Create Facelandmarker Object
# base_options = python.BaseOptions(model_asset_path=base_model)
# VisionRunningMode = mp.tasks.vision.RunningMode
# options = vision.FaceLandmarkerOptions(
#     base_options=base_options,
#     num_faces=1,
#     running_mode = VisionRunningMode.IMAGE,
# )
# detector = vision.FaceLandmarker.create_from_options(options)

In [175]:
# def get_cheek_rois(landmarks, image_shape):
#     h, w, _ = image_shape
#     left_cheek_indices = [111, 121, 50, 142]
#     right_cheek_indices = [350, 340, 355, 280]

#     left_cheek_points = [(int(landmarks[idx].x * w), int(landmarks[idx].y * h)) for idx in left_cheek_indices]
#     right_cheek_points = [(int(landmarks[idx].x * w), int(landmarks[idx].y * h)) for idx in right_cheek_indices]

#     left_cheek_rect = (
#         min([pt[0] for pt in left_cheek_points]), min([pt[1] for pt in left_cheek_points]),
#         max([pt[0] for pt in left_cheek_points]), max([pt[1] for pt in left_cheek_points])
#     )
#     # print("Left Cheek Rect:", left_cheek_rect)
#     right_cheek_rect = (
#         min([pt[0] for pt in right_cheek_points]), min([pt[1] for pt in right_cheek_points]),
#         max([pt[0] for pt in right_cheek_points]), max([pt[1] for pt in right_cheek_points])
#     )
#     # print("Right Cheek Rect:", right_cheek_rect)

#     return left_cheek_rect, right_cheek_rect


In [176]:
# def extract_rgb_from_rect(rect, image):
#     x_min, y_min, x_max, y_max = rect
#     roi = image[y_min:y_max, x_min:x_max]
#     return roi

<!-- ## Preprocessing Signal -->

In [177]:
# def preprocess_ppg(signal, fs = 30):
#     """ Computes the Preprocessed PPG Signal, this steps include the following:
#         1. Moving Average Smoothing
#         2. Bandpass Filtering
        
#         Parameters:
#         ----------
#         signal (numpy array): 
#             The PPG Signal to be preprocessed
#         fs (float): 
#             The Sampling Frequency of the Signal
            
#         Returns:
#         --------
#         numpy array: 
#             The Preprocessed PPG Signal
    
#     """ 

#     # Moving Average Smoothing
#     window = int(fs * 0.15)  # 150ms window
#     smoothed_signal = np.convolve(signal, np.ones(window)/window, mode='same')

#     b, a = scipy.signal.butter(2, [0.5, 2.5], btype='band', fs=fs)
#     filtered = scipy.signal.filtfilt(b, a, smoothed_signal)
    
#     # Additional lowpass to remove high-frequency noise
#     b2, a2 = scipy.signal.butter(3, 2.5, btype='low', fs=fs)
#     filtered = scipy.signal.filtfilt(b2, a2, filtered)
    
#     # Moving average smoothing
#     window = int(fs * 0.15)  # 150ms window
#     filtered_signal = np.convolve(filtered, np.ones(window)/window, mode='same')

#     # Normalize the signal
#     normalized_signal = (filtered_signal - np.min(filtered_signal)) / (np.max(filtered_signal) - np.min(filtered_signal))


#     return normalized_signal


<!-- ## HRV - v0.1

Extraction the RGB Signal over the entire frame image -->

In [178]:
# """ RGB Signal Variables

#     Variables to store the RGB signals from the left and right cheeks,
#     as well as the combined signal from both cheeks.

# """
# ## Utils
# fs = 30 # Sampling rate in Hz for RPPG signal


# # X should be increasing, y should be decreasing
# # w should be decreasing, h should be decreasing to fit the box into the face

# # Fixed adjustment values (in pixels)
# margin_x = 10  # Adjust horizontally
# scaling_factor = 0.8

In [179]:
# """
#     Since the base folders path is around "datasets/subjects, we will create a target folders processed
# """
# def get_image_roi(folders, image_file):
#     target_folders = "/processed"
#     rgb_folder = folders + "/rgb"

#     ## create the target folder
#     if not os.path.exists(folders + target_folders):
#         os.makedirs(folders + target_folders)
        
#     video_files = sorted(os.listdir(rgb_folder))

#     ## Get the first image
#     image_path = os.path.join(rgb_folder, video_files[0])
#     image = cv2.imread(image_path)

#     ## ===========================================
    
#     ## Get the image_path 
#     image_path = os.path.join(rgb_folder, video_files[0])
#     image = cv2.imread(image_path)

#     ## Convert the image to RGB
#     image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
#     mp_image = mp.Image(
#         image_format=mp.ImageFormat.SRGB,
#         data=image_rgb
#     )

#     ## Get the landkmarks
#     result = detector.detect(mp_image)

#     if result.detections:
#         print("Face Detected")
#         for detection in result.detections:

#             # Get the Bounding box
#             bboxC = detection.bounding_box
#             x, y, w, h = bboxC.origin_x, bboxC.origin_y, bboxC.width, bboxC.height

#             new_x = int(x + margin_x)

#             new_w = int(w * scaling_factor)
#             new_h = int(h * scaling_factor)

#             ## Draw the rectangle
#             cv2.rectangle(image_rgb, (new_x, y), (new_x + new_w, y + new_h), (0, 255, 0), 2)
    
#     # if result.face_landmarks:
#     #     ## Check if face detected
#     #     print("Face Detected")
#     #     for face_landmark in result.face_landmarks:
#     #         # Get cheek ROIs
#     #         left_cheek_rect, right_cheek_rect = get_cheek_rois(face_landmark, image_rgb.shape)

#     #         # Extract the RGB signals from the left and right cheeks
#     #         left_cheek_roi = extract_rgb_from_rect(left_cheek_rect, image_rgb)
#     #         right_cheek_roi = extract_rgb_from_rect(right_cheek_rect, image_rgb)

#     #         # print("Left Cheek ROI Shape:", left_cheek_roi.shape)
#     #         # print("Right Cheek ROI Shape:", right_cheek_roi.shape)

#     #         # Draw both cheek ROIs with rectangles
#     #         cv2.rectangle(image_rgb, (left_cheek_rect[0], left_cheek_rect[1]), (left_cheek_rect[2], left_cheek_rect[3]), (0, 255, 0), 2)
#     #         cv2.rectangle(image_rgb, (right_cheek_rect[0], right_cheek_rect[1]), (right_cheek_rect[2], right_cheek_rect[3]), (0, 255, 0), 2)
            
#     #         # Calculate mean pixel values for the RGB channels
#     #         left_cheek_rgb = cv2.mean(left_cheek_roi)[:3]
#     #         right_cheek_rgb = cv2.mean(right_cheek_roi)[:3]

#             ## Print the RGB value
#             # print("Left Cheek RGB:", left_cheek_rgb)
#             # print("Right Cheek RGB:", right_cheek_rgb)
#             # print("Combined RGB:", (combined_r, combined_g, combined_b))

#     ## Save the image into the Target Folder
#     target_path = os.path.join(folders + target_folders, image_file)
#     cv2.imwrite(target_path, cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR))

<!-- ## Saving the RGB -->

In [180]:
# def saving_roi_mediapipe(folders, image_file):

#     # Lists to store combined RGB values
#     combined_r_signal, combined_g_signal, combined_b_signal = [], [], []

#     target_folders = "/processed"
#     rgb_folder = folders + "/rgb"

#     ## create the target folder
#     if not os.path.exists(folders + target_folders):
#         os.makedirs(folders + target_folders)

#     # Load the video frames
#     video_files = sorted(os.listdir(rgb_folder))

#     # Load the face landmarks
#     for i in range(len(video_files) - 1): ## The last one is
#         # Load the image
#         image_path = os.path.join(rgb_folder, video_files[i])
#         image = cv2.imread(image_path)

#         ## Detect the face area using shape
#         h, w, _ = image.shape

#         ## Converting into RGB
#         image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#         ## Seting up the Mediapipe Image
#         mp_image = mp.Image(
#             image_format=mp.ImageFormat.SRGB,
#             data=image_rgb
#         )

#         # ## Get the landkmarks
#         result = detector.detect(mp_image)

#         if result.detections:
#             for detection in result.detections:

#                 ## Get the Bounding box
#                 bboxC = detection.bounding_box
#                 x, y, w, h = bboxC.origin_x, bboxC.origin_y, bboxC.width, bboxC.height

#                 new_x = int(x + margin_x)

#                 new_w = int(w * scaling_factor)
#                 new_h = int(h * scaling_factor)

#                 ## Get the ROI
#                 face_roi = image_rgb[y:y+new_h, new_x:new_x+new_w]

#                 ## Calculate the Mean
#                 mean_rgb = cv2.mean(face_roi)[:3]
                
#                 # Append the combined RGB values to the respective lists
#                 combined_r_signal.append(mean_rgb[0])
#                 combined_g_signal.append(mean_rgb[1])
#                 combined_b_signal.append(mean_rgb[2])

#         # Extract the face landmarks
#         # if result.face_landmarks:
#         #     for face_landmark in result.face_landmarks:
#         #         # Get cheek ROIs
#         #         left_cheek_rect, right_cheek_rect = get_cheek_rois(face_landmark, image_rgb.shape)

#         #         # Draw both cheek ROIs with rectangles
#         #         cv2.rectangle(image_rgb, (left_cheek_rect[0], left_cheek_rect[1]), (left_cheek_rect[2], left_cheek_rect[3]), (0, 255, 0), 2)
#         #         cv2.rectangle(image_rgb, (right_cheek_rect[0], right_cheek_rect[1]), (right_cheek_rect[2], right_cheek_rect[3]), (0, 255, 0), 2)

#         #         # Extract the left and right cheek ROIs
#         #         left_cheek_roi = extract_rgb_from_rect(left_cheek_rect, image_rgb)
#         #         right_cheek_roi = extract_rgb_from_rect(right_cheek_rect, image_rgb)

#         #         # Calculate mean pixel values for the RGB channels
#         #         left_cheek_rgb = cv2.mean(left_cheek_roi)[:3]
#         #         right_cheek_rgb = cv2.mean(right_cheek_roi)[:3]

#         #         # Combine and average the RGB values from both cheeks
#         #         combined_r = (left_cheek_rgb[0] + right_cheek_rgb[0]) / 2
#         #         combined_g = (left_cheek_rgb[1] + right_cheek_rgb[1]) / 2
#         #         combined_b = (left_cheek_rgb[2] + right_cheek_rgb[2]) / 2

#         #         # Append the combined RGB values to the respective lists
#         #         combined_r_signal.append(combined_r)
#         #         combined_g_signal.append(combined_g)
#         #         combined_b_signal.append(combined_b)


#     ## Plot the RGB path 
#     plt.figure(figsize=(15, 5))
#     plt.plot(combined_r_signal, 'r', label='Red')
#     plt.plot(combined_g_signal, 'g', label='Green')
#     plt.plot(combined_b_signal, 'b', label='Blue')
#     plt.title('RGB Signal')
#     plt.xlabel('Frame')
#     plt.ylabel('Intensity')
#     plt.legend()

#     ## Save the graph
#     plt.savefig(folders + target_folders + f"/{image_file}-rgb.png")
#     plt.close()

#     ## Calculating the RPPG signal
#     rgb_signals = np.array([combined_r_signal, combined_g_signal, combined_b_signal])
#     rgb_signals = rgb_signals.reshape(1, 3, -1)
#     rppg_signal = POS(rgb_signals, fps=30)
#     rppg_signal = rppg_signal.reshape(-1)

#     # Show the RPPG Signal
#     plt.figure(figsize=(20, 5))
#     plt.plot(rppg_signal, color='black')
#     plt.title('rPPG Signal')
#     plt.tight_layout()

#     ## Save the graph
#     plt.savefig(folders + target_folders + f"/{image_file}-rppg.png")
#     plt.close()

#     ## Save the RPPG Signal
#     rppg_path = folders + target_folders + f"/{image_file}-rppg.npy"
#     np.save(rppg_path, rppg_signal)


<!-- ## Preprocessed Signals
Save the Frequency Domain Pre / Post proccessing Signal -->

In [181]:
# def processing_signal(folders, image_file, fs=30):
#     target_folders = "/processed"
    
#     if not os.path.exists(folders + target_folders):
#         print("Subject can't be detected on Mediapipe, skipping")
#         return
    
#     ## Load the RPPG Signal
#     rppg_path = folders + target_folders + f"/{image_file}-rppg.npy"
#     rppg_signal = np.load(rppg_path)

#     ## Compute the Frequency Component (Before)
#     # Perform FFT on the RPPG signal
#     fft_result = np.fft.fft(rppg_signal)
#     fft_freqs = np.fft.fftfreq(len(rppg_signal), d=1/fs)

#     # Take the magnitude of the FFT result
#     fft_magnitude = np.abs(fft_result)

#     # Plot the FFT result
#     plt.figure(figsize=(12, 6))
#     plt.plot(fft_freqs[:len(fft_freqs)//2], fft_magnitude[:len(fft_magnitude)//2], color='blue')
#     plt.title('Frequency Spectrum of the RPPG Signal')
#     plt.xlabel('Frequency (Hz)')
#     plt.ylabel('Magnitude')
#     plt.grid(True)
    
#     ## Save the graph
#     plt.savefig(folders + target_folders + f"/{image_file}-fft-before.png")
#     plt.close()

#     ## Doing the preprocessing
#     ## Filter RPPG Signal
#     preprocessed_signal = preprocess_ppg(rppg_signal, fs=30)

#     ## Show the Filtered RPPG Signal
#     fig, ax = plt.subplots(2, 1, figsize=(20, 6))
#     ax[0].plot(rppg_signal, color='black')
#     ax[0].set_title('rPPG Signal - Before Filtering')
#     ax[1].plot(preprocessed_signal, color='black')
#     ax[1].set_title('rPPG Signal - After Filtering')
#     plt.tight_layout()
    
#     ## Save the graph
#     plt.savefig(folders + target_folders + f"/{image_file}-filtered.png")
#     plt.close()
            
#     ## Compute the Frequency Component (After)
#     # Perform FFT on the preprocessed RPPG signal
#     fft_result = np.fft.fft(preprocessed_signal)
#     fft_freqs = np.fft.fftfreq(len(preprocessed_signal), d=1/fs)

#     # Take the magnitude of the FFT result
#     fft_magnitude = np.abs(fft_result)

#     # Plot the FFT result
#     # Plot the FFT result
#     plt.figure(figsize=(12, 6))
#     plt.plot(fft_freqs[:len(fft_freqs)//2], fft_magnitude[:len(fft_magnitude)//2], color='blue')
#     plt.title('Frequency Spectrum of the RPPG Signal')
#     plt.xlabel('Frequency (Hz)')
#     plt.ylabel('Magnitude')
#     plt.grid(True)
    
#     ## Save the graph
#     plt.savefig(folders + target_folders + f"/{image_file}-fft-after.png")
#     plt.close()
    
#     return preprocessed_signal

<!-- ## Load the GT and Comparing the Graph -->


In [182]:
# def process_gt(folders, preprocessed_signal, image_file):
#     target_folders = "/processed"

#     ## Load the Ground Truth
#     gt_path = folders + f"/vernier/{image_file}_vernier_ecg.csv"
#     gt_ppg = pd.read_csv(gt_path, usecols=[1], header=None).values

#     ## Flatten the Signal
#     gt_ppg = gt_ppg.flatten()

#     ## Downsample to 30Hz
#     original_fs = 200
#     new_fs = 30

#     ## Downsampling the GT PPG Signal
#     gt_ppg = scipy.signal.resample(gt_ppg, int(len(gt_ppg) * new_fs / original_fs))

#     ## Preprocess the Ground Truth Signal
#     gt_ppg = preprocess_ppg(gt_ppg, fs=30)

#     ## Show the Filtered GT Signal
#     plt.figure(figsize=(20, 5))
#     plt.plot(gt_ppg, color='black')
#     plt.title('GT Signal - Before Filtering')
#     plt.tight_layout()
    
#     ## Save the graph
#     plt.savefig(folders + target_folders + f"/{image_file}-gt.png")
#     plt.close()

#     ## Comparing the Signals
#     ## Finds the Peaks from the Preprocessed Signal and Ground Truth
#     rppg_peaks, _  = scipy.signal.find_peaks(preprocessed_signal)
#     gt_peaks, _ = scipy.signal.find_peaks(gt_ppg)

#     ## Print the Peaks
#     print(f"Number of Peaks in rPPG Signal: {len(rppg_peaks)}")
#     print(f"Number of Peaks in Ground Truth: {len(gt_peaks)}")   

#     ## Plot the Signal
#     plt.figure(figsize=(20, 5)) 
#     plt.plot(preprocessed_signal, color='black', label='rPPG Signal')
#     plt.plot(rppg_peaks, preprocessed_signal[rppg_peaks], "x", color='red', label='rPPG Peaks')
#     plt.plot(gt_ppg, color='blue', label='Ground Truth')
#     plt.plot(gt_peaks, gt_ppg[gt_peaks], "x", color='green', label='Ground Truth Peaks')

#     ## Adjust the text position dynamically
#     y_max = max(max(preprocessed_signal), max(gt_ppg))  # Get the maximum y-value from both signals
#     plt.text(0.02 * len(preprocessed_signal), y_max * 0.9, 
#             f"rPPG Peaks: {len(rppg_peaks)}", color='red', fontsize=12, bbox=dict(facecolor='white', alpha=0.8))
#     plt.text(0.02 * len(preprocessed_signal), y_max * 0.8, 
#             f"GT Peaks: {len(gt_peaks)}", color='green', fontsize=12, bbox=dict(facecolor='white', alpha=0.8))

#     ## Add title, labels, and legend
#     plt.title('rPPG Signal vs Ground Truth')
#     plt.xlabel("Samples")
#     plt.ylabel('Amplitude')
#     plt.legend(['rPPG Signal', 'rPPG Peaks', 'Ground Truth', 'Ground Truth Peaks'])
    
#     ## Save the graph
#     plt.savefig(folders + target_folders + f"/{image_file}-comparison.png")
#     plt.close()


In [183]:
# ## mapping the folders
# base_path=f"../Physio-Itera/Dataset"

# ## Mapping the folders
# folders=os.listdir(base_path)

# ## Loop the folders /rgb and /vernier
# for folder in folders:
#     working_folder = base_path + f"/{folder}"
#     print(working_folder)
    
#     ## Check if the folder is a directory
#     if os.path.isdir(working_folder):
#         ## Get the RGB and Vernier Folders as the path

#         ## Check if the folder afa
#         if folder == "aqua2" or folder == :

#             subject = folder
#             rgb_folder = os.path.join(working_folder, "rgb")
#             vernier_folder = os.path.join(working_folder, "vernier")

#             ## Get the video files
#             get_image_roi(working_folder, f"{subject}-roi.png")

#             ## Convert the RPPG 
#             saving_roi_mediapipe(working_folder, f"{subject}")

#             ## Getting the pre-processed signal
#             preprocessed_signal = processing_signal(working_folder, f"{subject}")

#             ## Process the Ground Truth
#             process_gt(working_folder, preprocessed_signal, f"{subject}")


## Phase 2: Preprocessing Stuff
Since we have a the raw rppg value of either face landmark / face detection, I think let's vary the pre-processing stuff and see which one better

## First case - Ardoni case

Using the ardoni case filtering, we use the numpy files from the rppg and load the gt with different pre-processing steps

In [184]:
def preprocess_ppg(signal, fs = 30):
    """ Computes the Preprocessed PPG Signal, this steps include the following:
        1. Moving Average Smoothing
        2. Bandpass Filtering
        
        Parameters:
        ----------
        signal (numpy array): 
            The PPG Signal to be preprocessed
        fs (float): 
            The Sampling Frequency of the Signal
            
        Returns:
        --------
        numpy array: 
            The Preprocessed PPG Signal
    

    """ 

    # Additional lowpass tod remove high-frequency noise
    b, a = scipy.signal.butter(3, [0.9, 2.4], btype='band', fs=fs)
    filtered = scipy.signal.filtfilt(b, a, signal)
    
    # Normalize the signal
    normalized_signal = (filtered - np.min(filtered)) / (np.max(filtered) - np.min(filtered))


    return normalized_signal


In [185]:
def processing_signal(folders, image_file, fs=30):
    target_folders = "/processed" ## Face Detector

    export_folders = "/unos" ## First Case Export
    
    if not os.path.exists(folders + export_folders):
        os.makedirs(folders + export_folders)

    
    ## Load the RPPG Signal
    rppg_path = folders + target_folders + f"/{image_file}-rppg.npy"
    rppg_signal = np.load(rppg_path)

    ## Compute the Frequency Component (Before)
    # Perform FFT on the RPPG signal
    fft_result = np.fft.fft(rppg_signal)
    fft_freqs = np.fft.fftfreq(len(rppg_signal), d=1/fs)

    # Take the magnitude of the FFT result
    fft_magnitude = np.abs(fft_result)

    # Plot the FFT result
    plt.figure(figsize=(12, 6))
    plt.plot(fft_freqs[:len(fft_freqs)//2], fft_magnitude[:len(fft_magnitude)//2], color='blue')
    plt.title('Frequency Spectrum of the RPPG Signal')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude')
    plt.grid(True)
    
    ## Save the graph
    plt.savefig(folders + export_folders + f"/{image_file}-fft-before.png")
    plt.close()

    ## Doing the preprocessing
    ## Filter RPPG Signal
    preprocessed_signal = preprocess_ppg(rppg_signal, fs=30)

    ## Show the Filtered RPPG Signal
    fig, ax = plt.subplots(2, 1, figsize=(20, 6))
    ax[0].plot(rppg_signal, color='black')
    ax[0].set_title('rPPG Signal - Before Filtering')
    ax[1].plot(preprocessed_signal, color='black')
    ax[1].set_title('rPPG Signal - After Filtering')
    plt.tight_layout()
    
    ## Save the graph
    plt.savefig(folders + export_folders + f"/{image_file}-filtered.png")
    plt.close()
            
    ## Compute the Frequency Component (After)
    # Perform FFT on the preprocessed RPPG signal
    fft_result = np.fft.fft(preprocessed_signal)
    fft_freqs = np.fft.fftfreq(len(preprocessed_signal), d=1/fs)

    # Take the magnitude of the FFT result
    fft_magnitude = np.abs(fft_result)

    # Plot the FFT result
    # Plot the FFT result
    plt.figure(figsize=(12, 6))
    plt.plot(fft_freqs[:len(fft_freqs)//2], fft_magnitude[:len(fft_magnitude)//2], color='blue')
    plt.title('Frequency Spectrum of the RPPG Signal')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude')
    plt.grid(True)
    
    ## Save the graph
    plt.savefig(folders + export_folders + f"/{image_file}-fft-after.png")
    plt.close()
    
    return preprocessed_signal

In [186]:
def process_gt(folders, preprocessed_signal, image_file):

    export_folders = "/unos" ## First Case Export       

    ## Load the Ground Truth
    gt_path = folders + f"/vernier/{image_file}_vernier_ecg.csv"
    gt_ppg = pd.read_csv(gt_path, usecols=[1], header=None).values

    ## Flatten the Signal
    gt_ppg = gt_ppg.flatten()

    ## Downsample to 30Hz
    original_fs = 200
    new_fs = 30

    ## Downsampling the GT PPG Signal
    gt_ppg = scipy.signal.resample(gt_ppg, int(len(gt_ppg) * new_fs / original_fs))

    ## Preprocess the Ground Truth Signal
    gt_ppg = preprocess_ppg(gt_ppg, fs=30)

    ## Show the Filtered GT Signal
    plt.figure(figsize=(20, 5))
    plt.plot(gt_ppg, color='black')
    plt.title('GT Signal - Before Filtering')
    plt.tight_layout()
    
    ## Save the graph
    plt.savefig(folders + export_folders + f"/{image_file}-gt.png")
    plt.close()

    ## Comparing the Signals
    ## Finds the Peaks from the Preprocessed Signal and Ground Truth
    rppg_peaks, _  = scipy.signal.find_peaks(preprocessed_signal)
    gt_peaks, _ = scipy.signal.find_peaks(gt_ppg)

    ## Print the Peaks
    print(f"Number of Peaks in rPPG Signal: {len(rppg_peaks)}")
    print(f"Number of Peaks in Ground Truth: {len(gt_peaks)}")   

    ## Plot the Signal
    plt.figure(figsize=(20, 5)) 
    plt.plot(preprocessed_signal, color='black', label='rPPG Signal')
    plt.plot(rppg_peaks, preprocessed_signal[rppg_peaks], "x", color='red', label='rPPG Peaks')
    plt.plot(gt_ppg, color='blue', label='Ground Truth')
    plt.plot(gt_peaks, gt_ppg[gt_peaks], "x", color='green', label='Ground Truth Peaks')

    ## Adjust the text position dynamically
    y_max = max(max(preprocessed_signal), max(gt_ppg))  # Get the maximum y-value from both signals
    plt.text(0.02 * len(preprocessed_signal), y_max * 0.9, 
            f"rPPG Peaks: {len(rppg_peaks)}", color='red', fontsize=12, bbox=dict(facecolor='white', alpha=0.8))
    plt.text(0.02 * len(preprocessed_signal), y_max * 0.8, 
            f"GT Peaks: {len(gt_peaks)}", color='green', fontsize=12, bbox=dict(facecolor='white', alpha=0.8))

    ## Add title, labels, and legend
    plt.title('rPPG Signal vs Ground Truth')
    plt.xlabel("Samples")
    plt.ylabel('Amplitude')
    plt.legend(['rPPG Signal', 'rPPG Peaks', 'Ground Truth', 'Ground Truth Peaks'])
    
    ## Save the graph
    plt.savefig(folders + export_folders + f"/{image_file}-comparison.png")
    plt.close()


In [187]:
## mapping the folders
base_path=f"../Physio-Itera/Dataset"

## Mapping the folders
folders=os.listdir(base_path)

## Loop the folders /rgb and /vernier
for folder in folders:
    working_folder = base_path + f"/{folder}"
    print(working_folder)
    
    ## Check if the folder is a directory
    if os.path.isdir(working_folder):
        ## Get the RGB and Vernier Folders as the path

        ## Check if the folder afa
        if folder != "afa6":

            subject = folder
            rgb_folder = os.path.join(working_folder, "rgb")
            vernier_folder = os.path.join(working_folder, "vernier")

            ## Getting the pre-processed signal
            preprocessed_signal = processing_signal(working_folder, f"{subject}")

            ## Process the Ground Truth
            process_gt(working_folder, preprocessed_signal, f"{subject}")


../Physio-Itera/Dataset/ades2
Number of Peaks in rPPG Signal: 88
Number of Peaks in Ground Truth: 88
../Physio-Itera/Dataset/ades6
Number of Peaks in rPPG Signal: 82
Number of Peaks in Ground Truth: 84
../Physio-Itera/Dataset/adin2
Number of Peaks in rPPG Signal: 91
Number of Peaks in Ground Truth: 81
../Physio-Itera/Dataset/adin6
Number of Peaks in rPPG Signal: 92
Number of Peaks in Ground Truth: 83
../Physio-Itera/Dataset/afa2
Number of Peaks in rPPG Signal: 103
Number of Peaks in Ground Truth: 110
../Physio-Itera/Dataset/afa6
../Physio-Itera/Dataset/agus2
Number of Peaks in rPPG Signal: 111
Number of Peaks in Ground Truth: 116
../Physio-Itera/Dataset/agus6
Number of Peaks in rPPG Signal: 93
Number of Peaks in Ground Truth: 106
../Physio-Itera/Dataset/aice2
Number of Peaks in rPPG Signal: 120
Number of Peaks in Ground Truth: 134
../Physio-Itera/Dataset/aice6
Number of Peaks in rPPG Signal: 99
Number of Peaks in Ground Truth: 94
../Physio-Itera/Dataset/alana2
Number of Peaks in rPPG S

## Second case - using Chabysev filtering

Using the case filtering, we use the numpy files from the rppg and load the gt with different pre-processing steps

In [188]:
def preprocess_ppg(signal, fs = 30):
    """ Computes the Preprocessed PPG Signal, this steps include the following:
        1. Moving Average Smoothing
        2. Bandpass Filtering
        
        Parameters:
        ----------
        signal (numpy array): 
            The PPG Signal to be preprocessed
        fs (float): 
            The Sampling Frequency of the Signal
            
        Returns:
        --------
        numpy array: 
            The Preprocessed PPG Signal
    

    """ 
    b, a = scipy.signal.cheby2(4, 40, [0.9, 2.4], btype='band', fs=fs)

    # Additional lowpass tod remove high-frequency noise
    filtered = scipy.signal.filtfilt(b, a, signal)
    
    # Normalize the signal
    normalized_signal = (filtered - np.min(filtered)) / (np.max(filtered) - np.min(filtered))


    return normalized_signal


In [189]:
def processing_signal(folders, image_file, fs=30):
    target_folders = "/processed" ## Face Detector

    export_folders = "/duos" ## Second Case Export
    
    if not os.path.exists(folders + export_folders):
        os.makedirs(folders + export_folders)

    
    ## Load the RPPG Signal
    rppg_path = folders + target_folders + f"/{image_file}-rppg.npy"
    rppg_signal = np.load(rppg_path)

    ## Compute the Frequency Component (Before)
    # Perform FFT on the RPPG signal
    fft_result = np.fft.fft(rppg_signal)
    fft_freqs = np.fft.fftfreq(len(rppg_signal), d=1/fs)

    # Take the magnitude of the FFT result
    fft_magnitude = np.abs(fft_result)

    # Plot the FFT result
    plt.figure(figsize=(12, 6))
    plt.plot(fft_freqs[:len(fft_freqs)//2], fft_magnitude[:len(fft_magnitude)//2], color='blue')
    plt.title('Frequency Spectrum of the RPPG Signal')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude')
    plt.grid(True)
    
    ## Save the graph
    plt.savefig(folders + export_folders + f"/{image_file}-fft-before.png")
    plt.close()

    ## Doing the preprocessing
    ## Filter RPPG Signal
    preprocessed_signal = preprocess_ppg(rppg_signal, fs=30)

    ## Show the Filtered RPPG Signal
    fig, ax = plt.subplots(2, 1, figsize=(20, 6))
    ax[0].plot(rppg_signal, color='black')
    ax[0].set_title('rPPG Signal - Before Filtering')
    ax[1].plot(preprocessed_signal, color='black')
    ax[1].set_title('rPPG Signal - After Filtering')
    plt.tight_layout()
    
    ## Save the graph
    plt.savefig(folders + export_folders + f"/{image_file}-filtered.png")
    plt.close()
            
    ## Compute the Frequency Component (After)
    # Perform FFT on the preprocessed RPPG signal
    fft_result = np.fft.fft(preprocessed_signal)
    fft_freqs = np.fft.fftfreq(len(preprocessed_signal), d=1/fs)

    # Take the magnitude of the FFT result
    fft_magnitude = np.abs(fft_result)

    # Plot the FFT result
    # Plot the FFT result
    plt.figure(figsize=(12, 6))
    plt.plot(fft_freqs[:len(fft_freqs)//2], fft_magnitude[:len(fft_magnitude)//2], color='blue')
    plt.title('Frequency Spectrum of the RPPG Signal')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude')
    plt.grid(True)
    
    ## Save the graph
    plt.savefig(folders + export_folders + f"/{image_file}-fft-after.png")
    plt.close()
    
    return preprocessed_signal

In [190]:
def process_gt(folders, preprocessed_signal, image_file):

    export_folders = "/duos" ## Second Case Export       

    ## Load the Ground Truth
    gt_path = folders + f"/vernier/{image_file}_vernier_ecg.csv"
    gt_ppg = pd.read_csv(gt_path, usecols=[1], header=None).values

    ## Flatten the Signal
    gt_ppg = gt_ppg.flatten()

    ## Downsample to 30Hz
    original_fs = 200
    new_fs = 30

    ## Downsampling the GT PPG Signal
    gt_ppg = scipy.signal.resample(gt_ppg, int(len(gt_ppg) * new_fs / original_fs))

    ## Preprocess the Ground Truth Signal
    gt_ppg = preprocess_ppg(gt_ppg, fs=30)

    ## Show the Filtered GT Signal
    plt.figure(figsize=(20, 5))
    plt.plot(gt_ppg, color='black')
    plt.title('GT Signal - Before Filtering')
    plt.tight_layout()
    
    ## Save the graph
    plt.savefig(folders + export_folders + f"/{image_file}-gt.png")
    plt.close()

    ## Comparing the Signals
    ## Finds the Peaks from the Preprocessed Signal and Ground Truth
    rppg_peaks, _  = scipy.signal.find_peaks(preprocessed_signal)
    gt_peaks, _ = scipy.signal.find_peaks(gt_ppg)

    ## Print the Peaks
    print(f"Number of Peaks in rPPG Signal: {len(rppg_peaks)}")
    print(f"Number of Peaks in Ground Truth: {len(gt_peaks)}")   

    ## Plot the Signal
    plt.figure(figsize=(20, 5)) 
    plt.plot(preprocessed_signal, color='black', label='rPPG Signal')
    plt.plot(rppg_peaks, preprocessed_signal[rppg_peaks], "x", color='red', label='rPPG Peaks')
    plt.plot(gt_ppg, color='blue', label='Ground Truth')
    plt.plot(gt_peaks, gt_ppg[gt_peaks], "x", color='green', label='Ground Truth Peaks')

    ## Adjust the text position dynamically
    y_max = max(max(preprocessed_signal), max(gt_ppg))  # Get the maximum y-value from both signals
    plt.text(0.02 * len(preprocessed_signal), y_max * 0.9, 
            f"rPPG Peaks: {len(rppg_peaks)}", color='red', fontsize=12, bbox=dict(facecolor='white', alpha=0.8))
    plt.text(0.02 * len(preprocessed_signal), y_max * 0.8, 
            f"GT Peaks: {len(gt_peaks)}", color='green', fontsize=12, bbox=dict(facecolor='white', alpha=0.8))

    ## Add title, labels, and legend
    plt.title('rPPG Signal vs Ground Truth')
    plt.xlabel("Samples")
    plt.ylabel('Amplitude')
    plt.legend(['rPPG Signal', 'rPPG Peaks', 'Ground Truth', 'Ground Truth Peaks'])
    
    ## Save the graph
    plt.savefig(folders + export_folders + f"/{image_file}-comparison.png")
    plt.close()


In [191]:
## mapping the folders
base_path=f"../Physio-Itera/Dataset"

## Mapping the folders
folders=os.listdir(base_path)

## Loop the folders /rgb and /vernier
for folder in folders:
    working_folder = base_path + f"/{folder}"
    print(working_folder)
    
    ## Check if the folder is a directory
    if os.path.isdir(working_folder):
        ## Get the RGB and Vernier Folders as the path

        ## Check if the folder afa
        if folder != "afa6":

            subject = folder
            rgb_folder = os.path.join(working_folder, "rgb")
            vernier_folder = os.path.join(working_folder, "vernier")

            ## Getting the pre-processed signal
            preprocessed_signal = processing_signal(working_folder, f"{subject}")

            ## Process the Ground Truth
            process_gt(working_folder, preprocessed_signal, f"{subject}")


../Physio-Itera/Dataset/ades2
Number of Peaks in rPPG Signal: 81
Number of Peaks in Ground Truth: 88
../Physio-Itera/Dataset/ades6
Number of Peaks in rPPG Signal: 77
Number of Peaks in Ground Truth: 84
../Physio-Itera/Dataset/adin2
Number of Peaks in rPPG Signal: 77
Number of Peaks in Ground Truth: 81
../Physio-Itera/Dataset/adin6
Number of Peaks in rPPG Signal: 78
Number of Peaks in Ground Truth: 83
../Physio-Itera/Dataset/afa2
Number of Peaks in rPPG Signal: 97
Number of Peaks in Ground Truth: 110
../Physio-Itera/Dataset/afa6
../Physio-Itera/Dataset/agus2
Number of Peaks in rPPG Signal: 99
Number of Peaks in Ground Truth: 116
../Physio-Itera/Dataset/agus6
Number of Peaks in rPPG Signal: 77
Number of Peaks in Ground Truth: 106
../Physio-Itera/Dataset/aice2
Number of Peaks in rPPG Signal: 94
Number of Peaks in Ground Truth: 95
../Physio-Itera/Dataset/aice6
Number of Peaks in rPPG Signal: 83
Number of Peaks in Ground Truth: 83
../Physio-Itera/Dataset/alana2
Number of Peaks in rPPG Signa

## First case - Chabesev v0.2 case

Using case filtering, we use the numpy files from the rppg and load the gt with different pre-processing steps

In [192]:
def preprocess_ppg(signal, fs = 30):
    """ Computes the Preprocessed PPG Signal, this steps include the following:
        1. Moving Average Smoothing
        2. Bandpass Filtering
        
        Parameters:
        ----------
        signal (numpy array): 
            The PPG Signal to be preprocessed
        fs (float): 
            The Sampling Frequency of the Signal
            
        Returns:
        --------
        numpy array: 
            The Preprocessed PPG Signal
    

    """ 

    # Additional lowpass tod remove high-frequency noise
    b, a = scipy.signal.cheby2(3, 60, [0.9, 2.4], btype='band', fs=fs)
    # b, a = scipy.signal.cheby2(3, 60, [0.8, 2.7], btype='band', fs=fs)

    filtered = scipy.signal.filtfilt(b, a, signal)
    
    # Normalize the signal
    normalized_signal = (filtered - np.min(filtered)) / (np.max(filtered) - np.min(filtered))


    return normalized_signal


In [193]:
def processing_signal(folders, image_file, fs=30):
    target_folders = "/processed" ## Face Detector

    export_folders = "/tres" ## First Case Export
    
    if not os.path.exists(folders + export_folders):
        os.makedirs(folders + export_folders)

    
    ## Load the RPPG Signal
    rppg_path = folders + target_folders + f"/{image_file}-rppg.npy"
    rppg_signal = np.load(rppg_path)

    ## Compute the Frequency Component (Before)
    # Perform FFT on the RPPG signal
    fft_result = np.fft.fft(rppg_signal)
    fft_freqs = np.fft.fftfreq(len(rppg_signal), d=1/fs)

    # Take the magnitude of the FFT result
    fft_magnitude = np.abs(fft_result)

    # Plot the FFT result
    plt.figure(figsize=(12, 6))
    plt.plot(fft_freqs[:len(fft_freqs)//2], fft_magnitude[:len(fft_magnitude)//2], color='blue')
    plt.title('Frequency Spectrum of the RPPG Signal')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude')
    plt.grid(True)
    
    ## Save the graph
    plt.savefig(folders + export_folders + f"/{image_file}-fft-before.png")
    plt.close()

    ## Doing the preprocessing
    ## Filter RPPG Signal
    preprocessed_signal = preprocess_ppg(rppg_signal, fs=30)

    ## Show the Filtered RPPG Signal
    fig, ax = plt.subplots(2, 1, figsize=(20, 6))
    ax[0].plot(rppg_signal, color='black')
    ax[0].set_title('rPPG Signal - Before Filtering')
    ax[1].plot(preprocessed_signal, color='black')
    ax[1].set_title('rPPG Signal - After Filtering')
    plt.tight_layout()
    
    ## Save the graph
    plt.savefig(folders + export_folders + f"/{image_file}-filtered.png")
    plt.close()
            
    ## Compute the Frequency Component (After)
    # Perform FFT on the preprocessed RPPG signal
    fft_result = np.fft.fft(preprocessed_signal)
    fft_freqs = np.fft.fftfreq(len(preprocessed_signal), d=1/fs)

    # Take the magnitude of the FFT result
    fft_magnitude = np.abs(fft_result)

    # Plot the FFT result
    # Plot the FFT result
    plt.figure(figsize=(12, 6))
    plt.plot(fft_freqs[:len(fft_freqs)//2], fft_magnitude[:len(fft_magnitude)//2], color='blue')
    plt.title('Frequency Spectrum of the RPPG Signal')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude')
    plt.grid(True)
    
    ## Save the graph
    plt.savefig(folders + export_folders + f"/{image_file}-fft-after.png")
    plt.close()
    
    return preprocessed_signal

In [194]:
def process_gt(folders, preprocessed_signal, image_file):

    export_folders = "/tres"     

    ## Load the Ground Truth
    gt_path = folders + f"/vernier/{image_file}_vernier_ecg.csv"
    gt_ppg = pd.read_csv(gt_path, usecols=[1], header=None).values

    ## Flatten the Signal
    gt_ppg = gt_ppg.flatten()

    ## Downsample to 30Hz
    original_fs = 200
    new_fs = 30

    ## Downsampling the GT PPG Signal
    gt_ppg = scipy.signal.resample(gt_ppg, int(len(gt_ppg) * new_fs / original_fs))

    ## Preprocess the Ground Truth Signal
    gt_ppg = preprocess_ppg(gt_ppg, fs=30)

    ## Show the Filtered GT Signal
    plt.figure(figsize=(20, 5))
    plt.plot(gt_ppg, color='black')
    plt.title('GT Signal - Before Filtering')
    plt.tight_layout()
    
    ## Save the graph
    plt.savefig(folders + export_folders + f"/{image_file}-gt.png")
    plt.close()

    ## Comparing the Signals
    ## Finds the Peaks from the Preprocessed Signal and Ground Truth
    rppg_peaks, _  = scipy.signal.find_peaks(preprocessed_signal)
    gt_peaks, _ = scipy.signal.find_peaks(gt_ppg)

    ## Print the Peaks
    print(f"Number of Peaks in rPPG Signal: {len(rppg_peaks)}")
    print(f"Number of Peaks in Ground Truth: {len(gt_peaks)}")   

    ## Plot the Signal
    plt.figure(figsize=(20, 5)) 
    plt.plot(preprocessed_signal, color='black', label='rPPG Signal')
    plt.plot(rppg_peaks, preprocessed_signal[rppg_peaks], "x", color='red', label='rPPG Peaks')
    plt.plot(gt_ppg, color='blue', label='Ground Truth')
    plt.plot(gt_peaks, gt_ppg[gt_peaks], "x", color='green', label='Ground Truth Peaks')

    ## Adjust the text position dynamically
    y_max = max(max(preprocessed_signal), max(gt_ppg))  # Get the maximum y-value from both signals
    plt.text(0.02 * len(preprocessed_signal), y_max * 0.9, 
            f"rPPG Peaks: {len(rppg_peaks)}", color='red', fontsize=12, bbox=dict(facecolor='white', alpha=0.8))
    plt.text(0.02 * len(preprocessed_signal), y_max * 0.8, 
            f"GT Peaks: {len(gt_peaks)}", color='green', fontsize=12, bbox=dict(facecolor='white', alpha=0.8))

    ## Add title, labels, and legend
    plt.title('rPPG Signal vs Ground Truth')
    plt.xlabel("Samples")
    plt.ylabel('Amplitude')
    plt.legend(['rPPG Signal', 'rPPG Peaks', 'Ground Truth', 'Ground Truth Peaks'])
    
    ## Save the graph
    plt.savefig(folders + export_folders + f"/{image_file}-comparison.png")
    plt.close()


In [195]:
## mapping the folders
base_path=f"../Physio-Itera/Dataset"

## Mapping the folders
folders=os.listdir(base_path)

## Loop the folders /rgb and /vernier
for folder in folders:
    working_folder = base_path + f"/{folder}"
    print(working_folder)
    
    ## Check if the folder is a directory
    if os.path.isdir(working_folder):
        ## Get the RGB and Vernier Folders as the path

        ## Check if the folder afa
        if folder != "afa6":

            subject = folder
            rgb_folder = os.path.join(working_folder, "rgb")
            vernier_folder = os.path.join(working_folder, "vernier")

            ## Getting the pre-processed signal
            preprocessed_signal = processing_signal(working_folder, f"{subject}")

            ## Process the Ground Truth
            process_gt(working_folder, preprocessed_signal, f"{subject}")


../Physio-Itera/Dataset/ades2
Number of Peaks in rPPG Signal: 85
Number of Peaks in Ground Truth: 88
../Physio-Itera/Dataset/ades6
Number of Peaks in rPPG Signal: 89
Number of Peaks in Ground Truth: 84
../Physio-Itera/Dataset/adin2
Number of Peaks in rPPG Signal: 86
Number of Peaks in Ground Truth: 84
../Physio-Itera/Dataset/adin6
Number of Peaks in rPPG Signal: 87
Number of Peaks in Ground Truth: 83
../Physio-Itera/Dataset/afa2
Number of Peaks in rPPG Signal: 89
Number of Peaks in Ground Truth: 92
../Physio-Itera/Dataset/afa6
../Physio-Itera/Dataset/agus2
Number of Peaks in rPPG Signal: 87
Number of Peaks in Ground Truth: 94
../Physio-Itera/Dataset/agus6
Number of Peaks in rPPG Signal: 76
Number of Peaks in Ground Truth: 90
../Physio-Itera/Dataset/aice2
Number of Peaks in rPPG Signal: 90
Number of Peaks in Ground Truth: 89
../Physio-Itera/Dataset/aice6
Number of Peaks in rPPG Signal: 88
Number of Peaks in Ground Truth: 84
../Physio-Itera/Dataset/alana2
Number of Peaks in rPPG Signal: 